# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import plotly.express as px
import sqlite3

# Funções criadas

In [2]:
import unicodedata
import re

In [3]:
def normalizaColunas(columns):
    normalized_columns = []
    for column in columns:
        nfkd_form = unicodedata.normalize('NFKD', column)
        ascii_form = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
        ascii_form = re.sub(r'\W+', '_', ascii_form)
        ascii_form = ascii_form.lower()
        normalized_columns.append(ascii_form)
    return normalized_columns

# Importando as bases de dados

In [4]:
conn = sqlite3.connect('C:\\Users\\User\\Documents\\Web Data\\AP2\\3__bases_de_upload\\bancoSeries.db')
query = "SELECT * FROM seriesNotas"

In [5]:
df2 = pd.read_sql(query,conn)

In [6]:
# verificando o tamanho da base em linhas e colunas
df2.shape

(100, 7)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Título         100 non-null    object
 1   Estréia        100 non-null    object
 2   Final          100 non-null    object
 3   Nº episódios   100 non-null    object
 4   Classificação  97 non-null     object
 5   Nota           100 non-null    object
 6   Nº avaliações  100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB


In [8]:
df2.head()

,Título,Estréia,Final,Nº episódios,Classificação,Nota,Nº avaliações
0,Breaking Bad,2008,2013,62,16,"9,5","2,1 mi"
1,Planeta Terra II,2016,2016,6,10,"9,5",159 mil
2,Planeta Terra,2006,2006,11,Livre,"9,4",221 mil
3,Irmãos de Guerra,2001,2001,10,14,"9,4",529 mil
4,Chernobyl,2019,2019,5,16,"9,3",870 mil


# Primeiros tratamentos

## Resetando o index e eliminando os duplicados

In [9]:
df2 = df2.reset_index(drop=True)


## Editando os nomes das colunas

In [10]:
df2.columns

Index(['Título', 'Estréia', 'Final', 'Nº episódios', 'Classificação', 'Nota',
       'Nº avaliações'],
      dtype='object')

In [11]:
df2.columns = normalizaColunas(df2.columns)
df2.head()

,titulo,estreia,final,no_episodios,classificacao,nota,no_avaliacoes
0,Breaking Bad,2008,2013,62,16,"9,5","2,1 mi"
1,Planeta Terra II,2016,2016,6,10,"9,5",159 mil
2,Planeta Terra,2006,2006,11,Livre,"9,4",221 mil
3,Irmãos de Guerra,2001,2001,10,14,"9,4",529 mil
4,Chernobyl,2019,2019,5,16,"9,3",870 mil


### Nomes das colunas em minusculo

## Filtros

In [12]:
df2.columns

Index(['titulo', 'estreia', 'final', 'no_episodios', 'classificacao', 'nota',
       'no_avaliacoes'],
      dtype='object')

Alterando a coluna nota para o tipo float e trocando o ponto pela vírgula

In [13]:
df2['nota'] = df2['nota'].str.replace(',','.')
df2['nota'] = df2['nota'].astype(float)

In [14]:
df2['estreia'] = df2['estreia'].astype(int)

In [15]:
df2['no_episodios'] = df2['no_episodios'].astype(int)

In [16]:
df2['no_avaliacoes']

0      2,1 mi
1     159 mil
2     221 mil
3     529 mil
4     870 mil
       ...   
95     68 mil
96     88 mil
97    176 mil
98    226 mil
99     54 mil
Name: no_avaliacoes, Length: 100, dtype: object

In [17]:
df2['no_avaliacoes'] = df2['no_avaliacoes'].str.replace(' mil','000').str.replace(' mi','00000').str.replace(',','')
df2['no_avaliacoes'] = df2['no_avaliacoes'].astype(int)

In [18]:
df2['classificacao'].value_counts()

classificacao
16           30
Livre        17
14           14
12           12
18            7
TV-PG         5
TV-14         4
10            3
TV-MA         3
Not Rated     1
16+           1
Name: count, dtype: int64

In [19]:
url_referencia = 'https://www.primevideo.com/help/ref=atv_hp_nd_cnt?language=pt_PT&nodeId=GFGQU3WYEG6FSJFJ'
tabela_referencia = pd.read_html(url_referencia)[0]
tabela_referencia

,Valores da classificação local,Crianças (todas as idades),Crianças mais velhas (7 ou mais anos),Adolescentes (13 ou mais anos),Jovens adultos (16 ou mais anos),Adultos (18 ou mais anos)
0,MPAA(Filmes),G,PG,PG-13,NaN,NC-17
1,MPAA(Filmes),G,PG,PG-13,NaN,Sem classificação
2,MPAA(Filmes),G,PG,PG-13,NaN,Não classificado
3,MPAA(Filmes),G,PG,PG-13,NaN,Classificado
4,TVPG(TV),TV-G,TV-Y7,NaN,TV-14,TV-MA
5,TVPG(TV),TV-G,TV-Y7-FV,NaN,TV-14,TV-MA
6,TVPG(TV),TV-Y,TV-PG,NaN,TV-14,TV-MA


In [20]:
df2['classificacao'] = df2['classificacao'].str.replace('Livre','0').str.replace('TV-PG','10').str.replace('TV-14','14').str.replace('TV-MA','18').str.replace('Not Rated','0').str.replace('Not Rated','0').str.replace('16+','16')
df2['classificacao'] = df2['classificacao'].fillna(0)
df2['classificacao']  = df2['classificacao'].astype(int)

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   titulo         100 non-null    object 
 1   estreia        100 non-null    int32  
 2   final          100 non-null    object 
 3   no_episodios   100 non-null    int32  
 4   classificacao  100 non-null    int32  
 5   nota           100 non-null    float64
 6   no_avaliacoes  100 non-null    int32  
dtypes: float64(1), int32(4), object(2)
memory usage: 4.0+ KB


In [22]:
df2.describe().round(3).T

,count,mean,std,min,25%,50%,75%,max
estreia,100.0,2006.850,13.501,1959.0,1999.0,2011.0,2017.0,2024.0
no_episodios,100.0,67.140,128.513,3.0,11.0,27.5,65.0,1117.0
classificacao,100.0,11.520,6.325,0.0,10.0,14.0,16.0,18.0
nota,100.0,8.957,0.202,8.7,8.8,8.9,9.1,9.5
no_avaliacoes,100.0,241480.000,355125.856,12000.0,51000.0,121500.0,276750.0,2300000.0


In [23]:
nome_arquivo = 'notasSeriesTratado.csv'
df2.to_csv(f'../1_bases_tratadas/{nome_arquivo}', sep=';', index=False, encoding='UTF-8')

In [24]:
df2['classificacao']

0     16
1     10
2      0
3     14
4     16
      ..
95    18
96    16
97    16
98    12
99    16
Name: classificacao, Length: 100, dtype: int32

In [33]:
df2['classificacao'].unique()


array([16, 10,  0, 14, 12, 18])